# weibo 크롤링

## 크롤링 조건 설정

In [23]:
# 페이지 스크롤 다운 횟수 설정(숫자가 클수록 많은 페이지 크롤링)
num_scrolldown = 1
# 날짜 설정 : "설정 날짜" 포함, 이후 날짜 데이터 출력됨.
start_date = '2022-01-01'

In [24]:
from bs4 import BeautifulSoup as bs
import time
import random
from selenium import webdriver
import pandas as pd
from datetime import datetime

## 페이지 로그인

In [25]:
# 사이트 로그인
driver = webdriver.Chrome('chromedriver')
driver.get("https://weibo.com/login.php")
time.sleep(5)
driver.implicitly_wait(15)

# 창 최대화
driver.maximize_window()
time.sleep(1)

username = 'globalgg1@gg.go.kr'
password = 'zhsxpscm123!'

element_id = driver.find_element_by_xpath('//*[@id="loginname"]')
element_id.send_keys(username)
time.sleep(1)
element_password = driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[2]/div/input')
element_password.send_keys(password)

time.sleep(1.5)

##로그인버튼 클릭
driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[6]/a').click()
time.sleep(5)

<ipython-input-25-e33fdf7239dd>:14: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element_id = driver.find_element_by_xpath('//*[@id="loginname"]')
<ipython-input-25-e33fdf7239dd>:17: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element_password = driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[2]/div/input')
<ipython-input-25-e33fdf7239dd>:23: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[6]/a').click()


## 크롤링할 페이지 접속

In [26]:
# 페이지 접속
url = 'https://weibo.com/p/1001066011831795/home?from=page_100106&mod=TAB#place'
# webdriver 로드
driver.get(url)    
time.sleep(5)
driver.implicitly_wait(5)

## 페이지 스크롤 다운

In [27]:
# 페이지 스크롤 다운
for i in range(num_scrolldown):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep( random.uniform(1,3))
    driver.implicitly_wait(5)

In [28]:
# 페이지 크롤링
soup = bs(driver.page_source, 'lxml')
print("타이틀 : ", soup.title.text)

타이틀 :  京畿道官博的微博_微博


## 데이터 크롤링

### 타이틀 수집

In [ ]:
# 타이틀 수집
# 타이틀 첫번째 마침표까지만 가져오기
raw_contents = soup.find_all('div', class_='WB_text W_f14')
content_list = []
num = len(raw_contents)
for i in range(num):
    content = raw_contents[i].text.strip().split('。')[0]
    content_list.append(content)
    
print('원본 게시글 수 : ', len(raw_contents))
print('게시글 수 : ', len(content_list))

### 조회수 수집

In [ ]:
# 조회수 수집
raw_number_view = soup.find_all('i', class_='S_txt2')
number_view_list = []
for i in range(len(raw_number_view)):
    temp = raw_number_view[i].attrs['title']
    number_view = int(temp[9:-1])
    number_view_list.append(number_view)
    
print('조회수 리스트 원소수 : ', len(number_view_list))

### 좋아요, 공유수, 댓글 수 수집

In [ ]:
# 정보 수집
raw_info = soup.find_all('span', 'line S_line1')

In [ ]:
# 좋아요 수 수집
number_like_list = []
num_range = (len(raw_info) - 3) // 4 + 1
raw_info = soup.find_all('span', 'line S_line1')
for i in range(num_range):
    num = ( i + 1 ) * 4 - 1
    temp = raw_info[num]
    temp = temp.find_all('em')[-1].text
    number_like = int(temp)
    number_like_list.append(number_like)
    
print('좋아요 리스트 원소수 : ', len(number_like_list))

In [ ]:
# 공유 수 수집
number_share_list = []
num_range = (len(raw_info) - 3) // 4 + 1
raw_info = soup.find_all('span', 'line S_line1')
for i in range(num_range):
    num = ( i + 1 ) * 4 - 3
    temp = raw_info[num]
    temp = temp.find_all('em')[-1].text
    if '转发' in temp:
        number_share_list.append(0)
    else:
        number_like = int(temp)
        number_share_list.append(number_like)
        
print('공유수 리스트 원소수 : ', len(number_share_list))

In [ ]:
# 댓글수 수집
number_comment_list = []
num_range_like = (len(raw_info) - 3) // 4 + 1
raw_info = soup.find_all('span', 'line S_line1')
for i in range(num_range_like):
    num = ( i + 1 ) * 4 - 2
    temp = raw_info[num]
    temp = temp.find_all('em')[-1].text
    if '评论' in temp:
        number_comment_list.append(0)
    else:
        number_like = int(temp)
        number_comment_list.append(number_like)
        
print('댓글수 리스트 원소수 : ', len(number_comment_list))

### 게시글 날짜 수집

In [ ]:
# 게시글 날짜 수집
date_list = []
raw_date = soup.find_all('div', 'WB_from S_txt2')
for i in range(len(raw_date)):
    temp = raw_date[i].find('a', 'S_txt2')
    date = temp.attrs['title'][:10]
    date_list.append(date)
    
print('날짜 리스트 원소수 : ', len(date_list))

### 팔로워 수 수집

In [ ]:
# 팔로워 수 수집
number_follower_list = []
number_of_follower = soup.find_all('strong', 'W_f16')
num = int(number_of_follower[1].text)
number_follower_list.append(num)
for i in range(len(date_list)-1):
    number_follower_list.append(0)
    
print('팔로워 수 리스트 원소수 : ', len(number_follower_list))

### 게시글 링크 수집

In [29]:
# 링크 수집
link_list = []
raw_links = soup.find_all('div', 'WB_from S_txt2')
for link in raw_links:
    temp = 'https://weibo.com' + link.find('a' ,'S_txt2').attrs['href']
    link_list.append(temp)
    
print("링크 리스트 원소수 : ", len(link_list))

원본 게시글 수 :  30
게시글 수 :  30
조회수 리스트 원소수 :  30
좋아요 리스트 원소수 :  30
공유수 리스트 원소수 :  30
댓글수 리스트 원소수 :  30
날짜 리스트 원소수 :  30
팔로워 수 리스트 원소수 :  30
링크 리스트 원소수 :  30


## 크롤링 데이터 확인

In [ ]:
print('원본 게시글 수 : ', len(raw_contents))
print('게시글 수 : ', len(content_list))
print('조회수 리스트 원소수 : ', len(number_view_list))
print('좋아요 리스트 원소수 : ', len(number_like_list))
print('공유수 리스트 원소수 : ', len(number_share_list))
print('댓글수 리스트 원소수 : ', len(number_comment_list))
print('날짜 리스트 원소수 : ', len(date_list))
print('팔로워 수 리스트 원소수 : ', len(number_follower_list))
print("링크 리스트 원소수 : ", len(link_list))

In [ ]:
# chromedriver 종료
driver.quit()

## 데이터 셋 생성

In [30]:
# 데이터 프레임 생성
df = {'날짜':date_list, '제목':content_list, '조회수':number_view_list, '좋아요_수':number_like_list, '공유수':number_share_list, '댓글수':number_comment_list, '팔로워_수':number_follower_list, '링크':link_list}
data = pd.DataFrame(df)
data

,날짜,제목,조회수,좋아요_수,공유수,댓글수,팔로워_수,링크
0,2022-03-21,京畿道金浦市是自然和都市共存的和平生态旅游城市,2386,2,0,1,531609,https://weibo.com/6011831795/LkMfMbHdG?from=pa...
1,2022-03-19,现在快到了温暖的三月中旬,5787,4,1,0,0,https://weibo.com/6011831795/LkrLnvOrO?from=pa...
2,2022-03-17,京畿道坡州市是与北朝鲜开城市最邻近的城市,5080,3,1,0,0,https://weibo.com/6011831795/Lk97htiea?from=pa...
3,2022-03-15,寒冬已过，暖春到来,5187,5,1,2,0,https://weibo.com/6011831795/LjQGKB7ss?from=pa...
4,2022-03-12,华城八景， 济扶落潮和体验神奇海路 济扶岛这是京畿道华城市不可或缺的8个旅游景点之一的岛屿,7509,4,1,0,0,https://weibo.com/6011831795/LjnLVmwNf?from=pa...
5,2022-03-10,走感性时间的旅行地，平泽市风鸟村野游花园花园里吹着微笑风的这个地方是想和珍惜的人一起散步的花园,6659,5,0,2,0,https://weibo.com/6011831795/Lj5cR3FIu?from=pa...
6,2022-03-08,可以体验游艇和享受大海钓鱼的全谷港玛丽娜京畿道华城市全谷港玛丽娜有着可以享受以辽阔的大海为背...,7716,2,0,0,0,https://weibo.com/6011831795/LiNnAlUAv?from=pa...
7,2022-03-04,欧洲风格的购物美食街， 始兴市高级折扣店京畿道始兴市有高级折扣店是疲劳的都市人可以得到休息的...,8478,2,1,0,0,https://weibo.com/6011831795/LibISiazA?from=pa...
8,2022-03-02,京畿道东豆川市，在逍遥山相遇的历史中的雪花旅行京畿道东豆川市是 在地缘政治上具有重要意义的地方,12325,1,0,0,0,https://weibo.com/6011831795/LhSpqqt1c?from=pa...
9,2022-02-28,京畿道杨州Hot Place，在露营餐厅享受吧！从古至今以美丽风景和新鲜空气而有名的城市，京...,11490,1,0,0,0,https://weibo.com/6011831795/LhzKTe0Ht?from=pa...


## 데이터 셋. 날짜로 슬라이싱

In [31]:
# 데이터 셋 날짜로 슬라이싱
slicing_data = data[data['날짜'] >= start_date]
slicing_data

,날짜,제목,조회수,좋아요_수,공유수,댓글수,팔로워_수,링크
0,2022-03-21,京畿道金浦市是自然和都市共存的和平生态旅游城市,2386,2,0,1,531609,https://weibo.com/6011831795/LkMfMbHdG?from=pa...
1,2022-03-19,现在快到了温暖的三月中旬,5787,4,1,0,0,https://weibo.com/6011831795/LkrLnvOrO?from=pa...
2,2022-03-17,京畿道坡州市是与北朝鲜开城市最邻近的城市,5080,3,1,0,0,https://weibo.com/6011831795/Lk97htiea?from=pa...
3,2022-03-15,寒冬已过，暖春到来,5187,5,1,2,0,https://weibo.com/6011831795/LjQGKB7ss?from=pa...
4,2022-03-12,华城八景， 济扶落潮和体验神奇海路 济扶岛这是京畿道华城市不可或缺的8个旅游景点之一的岛屿,7509,4,1,0,0,https://weibo.com/6011831795/LjnLVmwNf?from=pa...
5,2022-03-10,走感性时间的旅行地，平泽市风鸟村野游花园花园里吹着微笑风的这个地方是想和珍惜的人一起散步的花园,6659,5,0,2,0,https://weibo.com/6011831795/Lj5cR3FIu?from=pa...
6,2022-03-08,可以体验游艇和享受大海钓鱼的全谷港玛丽娜京畿道华城市全谷港玛丽娜有着可以享受以辽阔的大海为背...,7716,2,0,0,0,https://weibo.com/6011831795/LiNnAlUAv?from=pa...
7,2022-03-04,欧洲风格的购物美食街， 始兴市高级折扣店京畿道始兴市有高级折扣店是疲劳的都市人可以得到休息的...,8478,2,1,0,0,https://weibo.com/6011831795/LibISiazA?from=pa...
8,2022-03-02,京畿道东豆川市，在逍遥山相遇的历史中的雪花旅行京畿道东豆川市是 在地缘政治上具有重要意义的地方,12325,1,0,0,0,https://weibo.com/6011831795/LhSpqqt1c?from=pa...
9,2022-02-28,京畿道杨州Hot Place，在露营餐厅享受吧！从古至今以美丽风景和新鲜空气而有名的城市，京...,11490,1,0,0,0,https://weibo.com/6011831795/LhzKTe0Ht?from=pa...


# 데이터 셋. 인덱스 내림차순 정렬

In [32]:
# 인덱스 내림차순 정렬
final_data = slicing_data.sort_index(ascending=False)
final_data

,날짜,제목,조회수,좋아요_수,공유수,댓글수,팔로워_수,링크
29,2022-01-01,🎊 新年快乐! 🎊祝你2022年与京畿道一起度过幸福的一年,11896,11,0,4,0,https://weibo.com/6011831795/L8IJk9tQd?from=pa...
28,2022-01-05,京畿道支援的异色创业趋势！充分利用共享厨房的 “G-创食学院”的运营共享厨房是指多个营业者共...,13823,3,0,0,0,https://weibo.com/6011831795/L9mdF9t5G?from=pa...
27,2022-01-06,美味的冬季京畿道传统市场一到冬天就会想起鲫鱼饼、烤红薯、鱼糕、甜饼等很多零食吧？请来访问具有...,15422,5,0,0,0,https://weibo.com/6011831795/L9wcfEBPu?from=pa...
26,2022-01-07,🍅京畿道光州市代表农作物番茄的重新诞生！酸酸甜甜的番茄辣椒酱同大韩民国代表酱料辣椒酱一样受到...,16073,4,0,1,0,https://weibo.com/6011831795/L9FbKuzBl?from=pa...
25,2022-01-13,京畿道为外国人居民定居支援的事业募集📣京畿道，支援外国人居民与高丽人同胞支援事业🔥在大韩民国...,15829,5,1,1,0,https://weibo.com/6011831795/LaAyPh9ZV?from=pa...
24,2022-01-15,MZ世代的元宇宙元宇宙图书馆，京畿道城南教育图书馆在京畿道有了元宇宙元宇宙图书馆,16737,3,0,0,0,https://weibo.com/6011831795/LaMlxesju?from=pa...
23,2022-01-17,京畿道平泽市，2021平泽世界文化周成功落下帷幕🎉连接世界的文化散策城市 🏃在京畿道平泽市举...,17931,5,1,1,0,https://weibo.com/6011831795/Lbck79XJz?from=pa...
22,2022-01-20,🎹 风乐响起来!! 🎺🎺在京畿道的博物馆都去过哪里？@@(学习历史最好的地方👍👍)👉京畿道博...,19327,3,1,0,0,https://weibo.com/6011831795/LbCqQmNzF?from=pa...
21,2022-01-24,主题公园，都去过哪里呢介绍一下位于京畿道的国内最大的汽车主题公园,19375,2,0,0,0,https://weibo.com/6011831795/LcdTRzXmm?from=pa...
20,2022-01-26,拥有很多世界文化遗产的美丽的水原华城2021年在华城举办了‘媒体艺术秀’,18656,3,1,1,0,https://weibo.com/6011831795/Lcyygmjkp?from=pa...


## 엑셀 파일로 출력

In [33]:
# 파일로 출력
file_name = str(datetime.today().strftime("%Y%m%d-%H%M%S"))
file_name = 'weibo_' + file_name + '.xlsx'
path = './crawling_data/weibo/'
final_data.to_excel(path+file_name, index=False, encoding='utf-8')

print(f"{file_name} 파일 생성 완료")

weibo_20220322-150058.xlsx 파일 생성 완료


# 코드 마지막